Projeto de Disciplina – Wikipédia: Lista de jogos licenciados do Nintendo Entertainment System
---

Universidade Federal de São Carlos

Curso: Bacharelado em Ciência da Computação de Sorocaba

Disciplina: Novas Tecnologias de Bancos de Dados

Professora: Profa. Dra. Sahudy Montenegro González

---

Grupo 10

Integrantes:
- Anne Mari Suenaga Sakai (822304)
- Felipe Pregnolatto Melo (822825)

---

Resumo:

Este projeto constrói um Data Lakehouse sobre os jogos licenciados do Nintendo Entertainment System (NES), utilizando dados extraídos da Wikipédia por meio de sua API e processados com BeautifulSoup. Os dados passam pelas camadas Bronze, Prata e Ouro até se consolidarem em tabelas estruturadas para análise. O sistema final adota um esquema estrela composto por uma tabela fato e três dimensões (desenvolvedora, publicadora e data), incluindo uma referência temporal para cada região de lançamento (Japão, América do Norte e região PAL). O objetivo é oferecer uma base organizada para consultas históricas, análises comparativas e estudos sobre padrões de parceria, distribuição de lançamentos e comportamento do mercado da época.


# 1 Camada Bronze
Na camada Bronze, realizamos a ingestão dos dados brutos diretamente da página oficial da Wikipédia contendo a lista completa de jogos licenciados de Nintendo Entertainment System (NES).

## 1.1 Extração do HTML pela API da Wikipédia
Como a API da Wikipédia não fornece os dados da tabela em formato estruturado, utilizamos a API apenas para baixar o HTML bruto, e então empregamos o BeautifulSoup para extrair manualmente as informações da tabela.

Utilizamos BeautifulSoup para navegar e interpretar o HTML, identificando a tabela de interesse (id="softwarelist").

In [0]:
# %pip install lxml

In [0]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, NavigableString, Tag

API_URL = "https://en.wikipedia.org/w/api.php"

params = {
    "action": "parse",
    "page": "List_of_Nintendo_Entertainment_System_games",
    "prop": "text",
    "section": 1,  # Licensed games
    "format": "json"
}

headers = {
    "User-Agent": "Databricks-NES-ETL/1.0 (anne.projeto@example.com)"
}

# -----------------------
# 1) BAIXAR HTML DA SEÇÃO
# -----------------------
response = requests.get(API_URL, params=params, headers=headers)
response.raise_for_status()
html = response.json()["parse"]["text"]["*"]

soup = BeautifulSoup(html, "html.parser")

table = soup.find("table", {"id": "softwarelist"})

## 1.2 Parsing manual do HTML

Após obter o HTML bruto via MediaWiki API, cada linha da tabela (`<tr>`) foi processada individualmente para extrair os dados com precisão. Essa etapa é necessária porque a estrutura da Wikipédia é altamente irregular, contendo misturas de texto, links, anotações, múltiplas entradas em uma mesma célula, além de formatação HTML inconsistente.

Os principais pontos que exigiram parsing manual foram:

### Título
Alguns jogos apresentam nomes alternativos ou formatações variadas, mas o título principal aparece sempre dentro da primeira tag `<i>`.  
Por isso, extraímos apenas a primeira ocorrência de `<i>`, que corresponde ao nome oficial do jogo.

### Publisher
A coluna de publicadoras é a mais heterogênea da tabela, podendo assumir diferentes formatos:

- texto simples;
- nomes dentro de links (`<a>`), com ou sem regiões associadas;
- múltiplas publicadoras separadas por `<br>`;
- regiões indicadas por `<sup>`;
- combinações entre texto puro e elementos `<a>`.

Essas variações impossibilitam extração automática padrão.

Para lidar com essa complexidade, foram aplicadas regras explícitas ao percorrer cada elemento da célula:

- identificação precisa de cada tipo de nó HTML (`<a>`, `<sup>`, `<br>`, `NavigableString`);
- associação correta entre publicadora e a região indicada no `<sup>` correspondente;
- geração de múltiplas linhas quando há mais de uma publicadora, garantindo que cada associação publicadora–região seja preservada.

Esse método garante que toda a multiplicidade de informações seja mantida de forma fiel à fonte original.

### Resultado do parsing
Após o processamento manual, obtém-se um dataset base contendo:

- **Titulo**
- **Desenvolvedora**
- **Publicadora**
- **Release_JP**
- **Release_NA**
- **Release_PAL**


In [0]:
# -----------------------
# 2) PROCESSAR MANUALMENTE CADA TR
# -----------------------
rows_processed = []

for tr in table.find_all("tr"):
    tds = tr.find_all("td")

    if len(tds) < 7:
        continue  # pular header ou linhas inválidas

    # ----------  TÍTULO  ----------
    i_tag = tds[0].find("i")
    title = i_tag.get_text(strip=True) if i_tag else tds[0].get_text(strip=True)


    # ----------  DEVELOPER ----------
    developer = tds[1].get_text(" ", strip=True)

    # ----------  PUBLISHER ----------
    publisher_td = tds[2]
    publisher_entries = []  # lista de (nome, região)
    current_name = None
    current_region = None

    for elem in publisher_td.children:

        # Caso 1: nome sem link (texto puro)
        if isinstance(elem, NavigableString):
            name = elem.strip()

            # ignorar separadores
            if name and name not in ["", ","]:
                current_name = name

        # Caso 2: nome com link
        elif isinstance(elem, Tag) and elem.name == "a":
            current_name = elem.get_text(strip=True)

        # Caso 3: região <sup>
        elif isinstance(elem, Tag) and elem.name == "sup":
            current_region = elem.get_text(strip=True)

            # Se temos nome + região → registrar
            if current_name:
                publisher_entries.append((current_name, current_region))
                current_name = None
                current_region = None

        # Caso 4: quebra de linha
        elif isinstance(elem, Tag) and elem.name == "br":
            continue

    # Caso especial: nome sem <sup>
    if current_name and not publisher_entries:
        publisher_entries.append((current_name, None))

    # ----------  RELEASES ----------
    release_jp = tds[4].get_text(strip=True) or "Unreleased"
    release_na = tds[5].get_text(strip=True) or "Unreleased"
    release_pal = tds[6].get_text(strip=True) or "Unreleased"

    release_map = {
        "JP": release_jp,
        "NA": release_na,
        "PAL": release_pal
    }

    # ----------  CRIAR LINHAS ----------
    for pub_name, region in publisher_entries:

        row = {
            "Titulo": title,
            "Desenvolvedora": developer,
            "Publicadora": pub_name,
            "Release_JP": "Unreleased",
            "Release_NA": "Unreleased",
            "Release_PAL": "Unreleased",
        }

        # Se não há região (nenhum <sup>)
        if region is None:
            row["Release_JP"] = release_jp
            row["Release_NA"] = release_na
            row["Release_PAL"] = release_pal

        else:
            # Pode ser "NA/PAL"
            for r in region.split("/"):
                if r in release_map:
                    row[f"Release_{r}"] = release_map[r]

        rows_processed.append(row)


In [0]:
df = pd.DataFrame(rows_processed)

display(df)
print(f"Total de jogos licenciados: {len(df)}")

Titulo,Desenvolvedora,Publicadora,Release_JP,Release_NA,Release_PAL
89 Dennō Kyūsei Uranai,Micronics,Jingukan Polaris,"December 10, 1988",Unreleased,Unreleased
2nd Super Robot Wars,Winkysoft,Banpresto,"December 29, 1991[6]",Unreleased,Unreleased
The 3-D Battles of WorldRunner,Square,Acclaim Entertainment,Unreleased[a],September 1987,Unreleased
4 Nin Uchi Mahjong,Hudson Soft,Nintendo,"November 2, 1984",Unreleased,Unreleased
8 Eyes,Thinking Rabbit,SETA,"September 27, 1988",Unreleased,Unreleased
8 Eyes,Thinking Rabbit,Taxan,Unreleased,January 1990,Unreleased
10-Yard Fight,Tose,Irem,"August 30, 1985",Unreleased,Unreleased
10-Yard Fight,Tose,Nintendo,Unreleased,"October 18, 1985","December 6, 1986"
720°,Beam Software,Mindscape,Unreleased,December 1989,Unreleased
1942,Micronics,Capcom,"December 11, 1985",November 1986,Unreleased


Total de jogos licenciados: 1590


--
# Camada Prata

Esta camada é responsável por padronizar os dados. Como a fonte de dados já teve a maioria dos dados padronizados, a única transformação necessária foi a das datas.
As datas podiam vir em três formas:
- Mês(por extenso) dia, Ano
- Mês(por extenso), Ano
- Ano

Então o código python cria novas colunas para adicionar:
- O ano
- O número de mês
- O dia
- O nome do mês em português

Em colunas separadas, sendo essas 4 colunas para cada uma das 3 regiões.
As regiões que estavam unreleased ficaram com o campo de ano nulo e o valor do mês e dia como -1.

In [0]:
if isinstance(df, pd.DataFrame):
    df = spark.createDataFrame(df)

from pyspark.sql.functions import (
    regexp_replace, regexp_extract, trim, when, lit, col
)

def padronizar_data(df, col_name):

    # 1. LIMPEZA DA COLUNA
    df = df.withColumn(col_name, regexp_replace(col_name, r"\[.*?\]", ""))   # remove [] vindo da extração dos elementos da Wikipédia

    # 2. EXTRAÇÃO DE PARTES DA DATA
    mes_en = regexp_extract(col(col_name), r"([A-Za-z]+)", 1)
    ano = regexp_extract(col(col_name), r"(\d{4})$", 1)
    dia = regexp_extract(col(col_name), r"[A-Za-z]+\s+(\d{1,2}),?\s+\d{4}",1)

    # 3. MÊS NUMÉRICO
    mes_num = (
        when(mes_en == "January", 1)
        .when(mes_en == "February", 2)
        .when(mes_en == "March", 3)
        .when(mes_en == "April", 4)
        .when(mes_en == "May", 5)
        .when(mes_en == "June", 6)
        .when(mes_en == "July", 7)
        .when(mes_en == "August", 8)
        .when(mes_en == "September", 9)
        .when(mes_en == "October", 10)
        .when(mes_en == "November", 11)
        .when(mes_en == "December", 12)
        .otherwise(None)
    )

    # 4. TRADUÇÃO EN para PT
    mes = (
        when(mes_en == "January", "Janeiro")
        .when(mes_en == "February", "Fevereiro")
        .when(mes_en == "March", "Março")
        .when(mes_en == "April", "Abril")
        .when(mes_en == "May", "Maio")
        .when(mes_en == "June", "Junho")
        .when(mes_en == "July", "Julho")
        .when(mes_en == "August", "Agosto")
        .when(mes_en == "September", "Setembro")
        .when(mes_en == "October", "Outubro")
        .when(mes_en == "November", "Novembro")
        .when(mes_en == "December", "Dezembro")
        .otherwise(None)
    )

    # 5. -1 QUANDO ESTIVER VAZIO
    dia_final = when(dia == "", lit(-1)).otherwise(dia)

    # 6. CRIAÇÃO DAS NOVAS COLUNAS
    df = df.withColumn(f"{col_name}_ano", ano)
    df = df.withColumn(f"{col_name}_mes_num", mes_num.cast("int"))
    df = df.withColumn(f"{col_name}_dia", dia_final)
    df = df.withColumn(f"{col_name}_mes", mes)

    return df


for region in ["Release_JP", "Release_NA", "Release_PAL"]:
    df = padronizar_data(df, region)


display(df)

Titulo,Desenvolvedora,Publicadora,Release_JP,Release_NA,Release_PAL,Release_JP_ano,Release_JP_mes_num,Release_JP_dia,Release_JP_mes,Release_NA_ano,Release_NA_mes_num,Release_NA_dia,Release_NA_mes,Release_PAL_ano,Release_PAL_mes_num,Release_PAL_dia,Release_PAL_mes
89 Dennō Kyūsei Uranai,Micronics,Jingukan Polaris,"December 10, 1988",Unreleased,Unreleased,1988,12,10,Dezembro,,null,-1,null,,null,-1,null
2nd Super Robot Wars,Winkysoft,Banpresto,"December 29, 1991",Unreleased,Unreleased,1991,12,29,Dezembro,,null,-1,null,,null,-1,null
The 3-D Battles of WorldRunner,Square,Acclaim Entertainment,Unreleased,September 1987,Unreleased,,null,-1,null,1987,9,-1,Setembro,,null,-1,null
4 Nin Uchi Mahjong,Hudson Soft,Nintendo,"November 2, 1984",Unreleased,Unreleased,1984,11,2,Novembro,,null,-1,null,,null,-1,null
8 Eyes,Thinking Rabbit,SETA,"September 27, 1988",Unreleased,Unreleased,1988,9,27,Setembro,,null,-1,null,,null,-1,null
8 Eyes,Thinking Rabbit,Taxan,Unreleased,January 1990,Unreleased,,null,-1,null,1990,1,-1,Janeiro,,null,-1,null
10-Yard Fight,Tose,Irem,"August 30, 1985",Unreleased,Unreleased,1985,8,30,Agosto,,null,-1,null,,null,-1,null
10-Yard Fight,Tose,Nintendo,Unreleased,"October 18, 1985","December 6, 1986",,null,-1,null,1985,10,18,Outubro,1986,12,6,Dezembro
720°,Beam Software,Mindscape,Unreleased,December 1989,Unreleased,,null,-1,null,1989,12,-1,Dezembro,,null,-1,null
1942,Micronics,Capcom,"December 11, 1985",November 1986,Unreleased,1985,12,11,Dezembro,1986,11,-1,Novembro,,null,-1,null


# Camada Ouro

## DimData

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col
from pyspark.sql import Row


cols = [
    col("Release_JP_ano").alias("ano"),
    col("Release_JP_mes_num").alias("mes_num"),
    col("Release_JP_dia").alias("dia"),
    col("Release_JP_mes").alias("mes"),
]


temp_datas_jp = df.select(*cols)

cols = [
    col("Release_NA_ano").alias("ano"),
    col("Release_NA_mes_num").alias("mes_num"),
    col("Release_NA_dia").alias("dia"),
    col("Release_NA_mes").alias("mes"),
]

temp_datas_na = df.select(*cols)

cols = [
    col("Release_PAL_ano").alias("ano"),
    col("Release_PAL_mes_num").alias("mes_num"),
    col("Release_PAL_dia").alias("dia"),
    col("Release_PAL_mes").alias("mes"),
]

temp_datas_pal = df.select(*cols)
temp_datas_jp = temp_datas_jp.filter(col("ano") != "")
temp_datas_na = temp_datas_na.filter(col("ano") != "")
temp_datas_pal = temp_datas_pal.filter(col("ano") != "")

#criando linha especial
DimData = temp_datas_jp.union(temp_datas_na).union(temp_datas_pal)

DimData = DimData.dropDuplicates()


window_spec = Window.orderBy(["ano", "mes_num", "dia"])

DimData = (
    DimData.withColumn(
        "pk_data",
        row_number().over(window_spec)
    )
    .select("pk_data", "ano", "mes_num", "dia", "mes")
)


#Linha especial
special_row = Row(pk_data=-1, ano=-1, mes_num=-1, dia=-1, mes="-1")
special_df = spark.createDataFrame([special_row])

DimData = DimData.union(special_df)


display(DimData)


pk_data,ano,mes_num,dia,mes
1,1983,7,15,Julho
2,1983,8,27,Agosto
3,1983,9,9,Setembro
4,1983,11,22,Novembro
5,1983,12,7,Dezembro
6,1983,12,12,Dezembro
7,1984,1,14,Janeiro
8,1984,2,2,Fevereiro
9,1984,2,18,Fevereiro
10,1984,4,21,Abril


## DimDesenvolvedoras

In [0]:


from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window_spec = Window.orderBy("Desenvolvedora")

dimDesenvolvedoras = (
    df.select(col("Desenvolvedora").alias("Desenvolvedora"))
    .distinct()
    .dropDuplicates()
)

dimDesenvolvedoras = (
    dimDesenvolvedoras.withColumn(
        "pk_dev",
        row_number().over(window_spec)
    )
    .select("pk_dev", "Desenvolvedora")
)

display(dimDesenvolvedoras)

pk_dev,Desenvolvedora
1,
2,A-Wave
3,A.I
4,AIM
5,ASCII
6,ASK
7,Activision
8,Advance Communication Company
9,Aicom
10,Aisystem Tokyo


## DimPublicadoras

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window_spec = Window.orderBy("Publicadora")

dimPublicadoras = (
    df.select(col("Publicadora").alias("Publicadora"))
    .distinct()
    .dropDuplicates()
)

dimPublicadoras = (
    dimPublicadoras.withColumn(
        "pk_pub",
        row_number().over(window_spec)
    )
    .select("pk_pub", "Publicadora")
)

display(dimPublicadoras)

pk_pub,Publicadora
1,A-Wave
2,ASCII
3,ASK
4,Absolute Entertainment
5,Acclaim Entertainment
6,Activision
7,Altron
8,American Sammy
9,American Softworks
10,American Softworks Corporation


## Juntando na tabela fato





In [0]:
from pyspark.sql.functions import lit, col, expr

df_base = (
    df
    .join(dimDesenvolvedoras, on="Desenvolvedora", how="left")
    .join(dimPublicadoras,   on="Publicadora",   how="left")
)



df_union = (
    df_base.select(
        col("Titulo"),col("pk_dev"), col("pk_pub"),
        lit("JP").alias("local"),
        col("Release_JP_ano").alias("ano"),
        col("Release_JP_mes_num").alias("mes_num"),
        col("Release_JP_dia").alias("dia")
    )
    .union(
        df_base.select(
             col("Titulo"),col("pk_dev"), col("pk_pub"),
            lit("NA").alias("local"),
            col("Release_NA_ano").alias("ano"),
            col("Release_NA_mes_num").alias("mes_num"),
            col("Release_NA_dia").alias("dia")
        )
    )
    .union(
        df_base.select(
            col("Titulo"),col("pk_dev"), col("pk_pub"),
            lit("PAL").alias("local"),
            col("Release_PAL_ano").alias("ano"),
            col("Release_PAL_mes_num").alias("mes_num"),
            col("Release_PAL_dia").alias("dia")
        )
    )
)

df_union = (
    df_union
        .withColumn("ano_str", col("ano").cast("string"))
        .withColumn("mes_num_str", col("mes_num").cast("string"))
        .withColumn("dia_str", col("dia").cast("string"))
)
df_union.printSchema()

df_union_clean = (
    df_union
        .withColumn("ano",
            when(col("ano_str") == "", None)
            .otherwise(col("ano_str")).cast("int")
        )
        .withColumn("mes_num",
            when(col("mes_num_str") == "", None)
            .otherwise(col("mes_num_str")).cast("int")
        )
        .withColumn("dia",
            when(col("dia_str") == "", None)
            .otherwise(col("dia_str")).cast("int")
        )
        .drop("ano_str", "mes_num_str", "dia_str")
)


df_union_clean.printSchema()

#display(df_union_clean)


df_union = df_union_clean

df_join = (
    df_union.join(
        DimData,
        on=[
            df_union.ano == DimData.ano,
            df_union.mes_num == DimData.mes_num,
            df_union.dia == DimData.dia
        ],
        how="left"
    )
    .select("Titulo", "pk_dev", "pk_pub", "local", "pk_data")

)


from pyspark.sql.functions import coalesce, lit, first

df_join = df_join.withColumn("pk_data", coalesce(col("pk_data"), lit(-1)))

#display(df_join)


JogosFato = (
    df_join
        .groupBy("Titulo", "pk_dev", "pk_pub")
        .pivot("local")
        .agg(first("pk_data"))
        .withColumnRenamed("JP",  "pk_data_JP")
        .withColumnRenamed("NA",  "pk_data_NA")
        .withColumnRenamed("PAL", "pk_data_PAL")
        .withColumn(
            "lugares_lançados",
            (col("pk_data_JP") != -1).cast("int")
            + (col("pk_data_NA") != -1).cast("int")
            + (col("pk_data_PAL") != -1).cast("int")
        )
)


display(JogosFato)





root
 |-- Titulo: string (nullable = true)
 |-- pk_dev: integer (nullable = true)
 |-- pk_pub: integer (nullable = true)
 |-- local: string (nullable = false)
 |-- ano: string (nullable = true)
 |-- mes_num: integer (nullable = true)
 |-- dia: long (nullable = true)
 |-- ano_str: string (nullable = true)
 |-- mes_num_str: string (nullable = true)
 |-- dia_str: string (nullable = true)

root
 |-- Titulo: string (nullable = true)
 |-- pk_dev: integer (nullable = true)
 |-- pk_pub: integer (nullable = true)
 |-- local: string (nullable = false)
 |-- ano: integer (nullable = true)
 |-- mes_num: integer (nullable = true)
 |-- dia: integer (nullable = true)



Titulo,pk_dev,pk_pub,pk_data_JP,pk_data_NA,pk_data_PAL,lugares_lançados
Advanced Dungeons & Dragons: Heroes of the Lance,147,95,573,-1,-1,1
Abadox: The Deadly Inner War,147,81,-1,479,-1,1
10-Yard Fight,250,58,52,-1,-1,1
Baseball,156,89,5,60,112,3
Bill Elliott's NASCAR Challenge,62,68,-1,571,-1,1
American Dream,38,26,432,-1,-1,1
Adventures of Lolo 2,84,43,465,479,-1,2
Alien 3,185,71,-1,740,-1,1
Aladdin,144,144,-1,-1,811,1
The Adventures of Rad Gravity,107,6,-1,543,592,2


## Salvando as tabelas

In [0]:
dimDesenvolvedoras.write.mode("overwrite").saveAsTable("DimDesenvolvedoras")
dimPublicadoras.write.mode("overwrite").saveAsTable("DimPublicadoras")
DimData.write.mode("overwrite").saveAsTable("DimData")
JogosFato.write.mode("overwrite").saveAsTable("JogoFato")

# Consultas

In [0]:
from pyspark.sql.functions import countDistinct
import matplotlib.pyplot as plt

#1. Desenvolvedoras que lançaram pelo menos 1 jogo em somente 1 região vs desenvolvedoras que somente lançaram em multi regiões


query_one_region = """
SELECT DISTINCT d.pk_dev
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
WHERE 
    (jf.pk_data_JP != -1 AND jf.pk_data_NA = -1 AND jf.pk_data_PAL = -1)
 OR (jf.pk_data_JP = -1 AND jf.pk_data_NA != -1 AND jf.pk_data_PAL = -1)
 OR (jf.pk_data_JP = -1 AND jf.pk_data_NA = -1 AND jf.pk_data_PAL != -1)
"""
devs_uma_regiao = spark.sql(query_one_region)

#Seleciona todos os desenvolvedores para tirar os que lançaram em uma região
all_devs = dimDesenvolvedoras.select("pk_dev").distinct()
devs_multi_region = all_devs.subtract(devs_one_region)


#Já que provavelmente são poucos, é interesante ver quais eles são
devs_multi_region_names = (
    devs_multi_region
    .join(dimDesenvolvedoras, on="pk_dev", how="left")
    .select("Desenvolvedora")
)
display(devs_multi_region_names)


#Pegue o número para mostrar na tabeça
count_one_region = devs_one_region.count()
count_multi_region = devs_multi_region.count()

plt.figure(figsize=(6,4))
plt.bar(["1 região", "Somente 2 ou mais"], [count_one_region, count_multi_region], color=["#1f77b4", "#ff7f0e"])
plt.ylabel("Quantidade de devs")
plt.title("Desenvolvedores: 1 região vs. Somente 2 ou mais")
plt.show()

#2.Comparar quantidade de desenvolvedores que lançaram 1 jogo somente em cada região entre 1986 e 1990

#So jp
query_only_jp = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano BETWEEN 1986 AND 1990
    AND dd_na.ano = -1
    AND dd_pal.ano = -1
"""
devs_only_jp = spark.sql(query_only_jp)

#na
query_only_na = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano = -1
    AND dd_na.ano BETWEEN 1986 AND 1990
    AND dd_pal.ano = -1
"""
devs_only_na = spark.sql(query_only_na)

#PAL
query_only_pal = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano = -1
    AND dd_na.ano = -1
    AND dd_pal.ano BETWEEN 1986 AND 1990
"""
devs_only_pal = spark.sql(query_only_pal)

plt.figure(figsize=(6,4))
plt.bar(
    ["JP", "NA", "PAL"],
    [devs_only_jp.count(), devs_only_na.count(), devs_only_pal.count()],
)
plt.ylabel("Quantidade de devs")
plt.title("Devs somente 1 região entre 1986 e 1990")
plt.show()



#So jp
query_only_jp = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano = 1990 AND dd_jp.mes_num = 9
    AND dd_na.ano = -1
    AND dd_pal.ano = -1
"""
devs_only_jp = spark.sql(query_only_jp)

#na
query_only_na = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano = -1
    AND dd_na.ano = 1990 AND dd_na.mes_num = 9
    AND dd_pal.ano = -1
"""
devs_only_na = spark.sql(query_only_na)

#PAL
query_only_pal = """
SELECT DISTINCT d.Desenvolvedora
FROM JogoFato jf
JOIN DimDesenvolvedoras d ON jf.pk_dev = d.pk_dev
JOIN DimData dd_jp ON jf.pk_data_JP = dd_jp.pk_data
LEFT JOIN DimData dd_na ON jf.pk_data_NA = dd_na.pk_data
LEFT JOIN DimData dd_pal ON jf.pk_data_PAL = dd_pal.pk_data
WHERE 
    dd_jp.ano = -1
    AND dd_na.ano = -1
    AND dd_pal.ano = 1990 AND dd_pal.mes_num = 9
"""
devs_only_pal = spark.sql(query_only_pal)

plt.figure(figsize=(6,4))
plt.bar(
    ["JP", "NA", "PAL"],
    [devs_only_jp.count(), devs_only_na.count(), devs_only_pal.count()],
    color = ["green"]
)
plt.ylabel("Quantidade de devs")
plt.title("Devs somente 1 região em setembro de 1990")
plt.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4585897971493064>, line 20
     18 #Seleciona todos os desenvolvedores para tirar os que lançaram em uma região
     19 all_devs = dimDesenvolvedoras.select("pk_dev").distinct()
---> 20 devs_multi_region = all_devs.subtract(devs_one_region)
     23 #Já que provavelmente são poucos, é interesante ver quais eles são
     24 devs_multi_region_names = (
     25     devs_multi_region
     26     .join(dimDesenvolvedoras, on="pk_dev", how="left")
     27     .select("Desenvolvedora")
     28 )

NameError: name 'devs_one_region' is not defined